In [ ]:
!pip install -U openpyxl selenium beautifulsoup4 lxml requests

In [ ]:
'''
注意事項:
下載對應的 ChromeDriver (web driver) 到程式檔案同一個目錄下後解壓縮，下載前記得對應版本編號。
連結: https://chromedriver.chromium.org/downloads

參考網頁:
[1] 臺大中文學報
http://www.cl.ntu.edu.tw/web/academic/journal.jsp
[2] sqlite3 --- SQLite 数据库 DB-API 2.0 接口模块
https://docs.python.org/zh-tw/3/library/sqlite3.html
'''


'''
匯入套件
'''
# HTML parser
from bs4 import BeautifulSoup as bs

# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

# 強制等待 (執行期間休息一下)
from time import sleep

# pretty-print
from pprint import pprint

# 隨機
from random import randint

# 計時
import time

# 整理 json 使用的工具
import json

# 執行 shell command 的時候用的
import os

# 取得錯誤訊息
import sys, traceback

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 正規表達式
import re

# 編碼
from urllib.parse import quote

# SQLite 資料庫
import sqlite3

# 存取 Excel 的工具
from openpyxl import load_workbook
from openpyxl import Workbook

# 取得系統時間的工具
from datetime import datetime

# 引入 hashlib 模組
import hashlib

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")             #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")        #最大化視窗
my_options.add_argument("--incognito")              #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

# 指定 chromedriver 檔案的路徑
driver_exec_path = './chromedriver.exe'

# 給 web driver 用的變數
driver = None

# 來源首頁
prefix_url = 'http://www.cl.ntu.edu.tw'
url = prefix_url + '/web/academic/journal.jsp'

# 指定 sheet name
folderName = sheetName = 'www_cl_ntu'

# 指定 json 檔名
jsonFileName = f'{folderName}.json'

# 建立儲存檔案用的資料夾
folderPath = f'./{folderName}'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

#預設下載路徑
my_options.add_experimental_option("prefs", {
    "download.default_directory": folderPath
})

# 放置爬取的資料
listData = []

# 分頁網址
listPage = []

In [ ]:
'''
函式
'''
# md5 (用來為每一筆資料建立唯一代號)
def md5(string):
    m = hashlib.md5()
    m.update(string.encode("utf-8"))
    return m.hexdigest()

# 初始化 Web Driver
def init():
    global driver
    # 使用 Chrome 的 WebDriver
    driver = webdriver.Chrome( 
        options = my_options, 
        executable_path = driver_exec_path
    )
    
# 走訪頁面
def visit():
    global driver
    
    try:
        # 走訪首頁
        driver.get(url)

        # 等待目標元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'div.numberPage_rightArea > div.numberPage_rightList')
            )
        )
        
    except TimeoutException as e:
        print('等待逾時: visit')
    
# 剖析元素資料
def parse():
    global driver
    
    try:       
        # 連續按下一頁，同時取得列表連結資訊
        while True:
            # 走訪列表
            for div in driver.find_elements(By.CSS_SELECTOR, 'div.AcademicPaperBg > div.PublishingList'):
                # 有 a 的 div，才需要走訪
                if len(div.find_elements(By.CSS_SELECTOR, 'div.PublishingTit > a[href]')) > 0:
                    # 取得超連結與文字
                    a_elm = div.find_element(By.CSS_SELECTOR, 'div.PublishingTit > a[href]')
                    strLink = a_elm.get_attribute('href')
                    strJournalTitle = a_elm.get_attribute('innerText')

                    # 出版日期
                    strPublishDate = div.find_element(By.CSS_SELECTOR, 'div.PublishingYearr').get_attribute('innerText')
                    strPublishDate = strPublishDate.strip()

                    # 系別
                    strDeptName = div.find_element(By.CSS_SELECTOR, 'div.PublishingListProfessor').get_attribute('innerText')
                    strDeptName = strDeptName.strip()

                    # 整理資料
                    listData.append({
                        "書名": strJournalTitle,
                        "連結": strLink,
                        "出版年": strPublishDate,
                        "系別": strDeptName
                    })

            # 等待
            sleep(1)
            
            # 若有下一頁，就按下去，沒有就離開迴圈
            if len(driver.find_elements(By.CSS_SELECTOR, 'div.numberPage_rightArea > div.numberPage_rightList')) > 0:
                # 取得下一頁跟最後一頁的元素(第一個是下一頁，第二個是最後一頁)
                driver.find_elements(By.CSS_SELECTOR, 'div.numberPage_rightArea > div.numberPage_rightList')[0].click()
            else:
                break
                
        # 逐個內頁走訪並取得資料
        for index, myDict in enumerate(listData):
            # 建立放置內頁期刊資料的變數
            if not 'sub' in listData[index]: listData[index]['sub'] = []
            
            # 等待
            sleep(1)
            
            # 走訪內頁
            driver.get(myDict['連結'])
            
            # 取得每一個 tr
            tr_elms = driver.find_elements(By.CSS_SELECTOR, 'table.MsoTableGrid > tbody > tr')
            
            # 初始化變數
            strJournalName = strPdfLink = strAuthor = ''
            
            # 有 pdf 連結的 tr 才進行資料抓取
            for tr in tr_elms:
                if len(tr.find_elements(By.CSS_SELECTOR, 'td a[href$=pdf]')) > 0:                   
                    # 篇名
                    a_elm = tr.find_element(By.CSS_SELECTOR, 'td a[href$=pdf]')
                    strJournalName = a_elm.get_attribute('innerText').strip()
                    
                    # pdf 連結
                    strPdfLink = a_elm.get_attribute('href')
                    
                    # 作者
                    strAuthor = tr.find_elements(By.CSS_SELECTOR, 'td')[1].get_attribute('innerText').strip()
                    
                    # 整理資料
                    listData[index]['sub'].append({
                        'id': md5(strPdfLink),
                        '篇名': strJournalName,
                        '檔案連結': strPdfLink,
                        '作者': strAuthor
                    })
            
    except TimeoutException as e:
        print('等待逾時: parse')
    
        
# 關閉瀏覽器
def close():
    global driver
    driver.quit()
        
# 儲存成 json
def saveJson():
    global listData
    with open(f"{folderPath}/{jsonFileName}", "w", encoding="utf-8") as file:
        file.write( json.dumps( listData, ensure_ascii=False, indent=4 ) )
    
# 儲存 .db
def saveDB():
    with open(f"{folderPath}/{jsonFileName}", "r", encoding="utf-8") as file:      
        # 取得 json 內容
        strJson = file.read()
        
        # 將 json 轉成 list
        listJson = json.loads(strJson)
        
    # 寫入對話記錄
    conn = sqlite3.connect(f"{folderPath}/{sheetName}.db")
    
    # 建立 cursor 物件
    cursor = conn.cursor()

    # 執行 SQL 語法
    try:
        # 查詢特定資料
        sql_query = f'''
        SELECT 1
        FROM journals
        WHERE id = ?
        '''
        
        # 寫入資料
        sql_insert = f'''
        INSERT INTO journals (
            id, publish_date, journal_title, journal_sub_title, author, 
            link, pdf_link, is_downloaded, created_at, updated_at
        ) VALUES ( 
            ?,?,?,?,?,
            ?,?,?,?,?
        )
        '''
        
        # 放置準備寫入的資料
        list_insert = []
        
        # 將 json 資料一筆一筆找出來
        for myDict in listJson:
            # 每一筆資料裡面有 sub 屬性，裡面有內頁的 pdf 連結等資訊，所以也要找出來
            for d in myDict['sub']:
                # 如果資料庫沒有這筆資料(透過 id)，則暫時將資料以 tuple 格式放到 list 當中
                if cursor.execute(sql_query, (d["id"],)).fetchone() == None:
                    list_insert.append((
                        d['id'],
                        myDict['出版年'],
                        myDict['書名'],
                        d['篇名'],
                        d['作者'],
                        myDict['連結'],
                        d['檔案連結'],            
                        0,
                        datetime.today().strftime("%Y-%m-%d %H-%M-%S"),
                        datetime.today().strftime("%Y-%m-%d %H-%M-%S")
                    ))
        
        # 新增資料到資料庫當中
        cursor.executemany(sql_insert, list_insert)
        
        # 執行 SQL 語法
        conn.commit()
    except sqlite3.Error as err: 
        # 回滾
        conn.rollback()

        # SQLite3 例外處理
        exc_type, exc_value, exc_tb = sys.exc_info()
        strErrorMsg = f'''SQLite error: {' '.join(err.args)}\n\n
        SQLite traceback: {traceback.format_exception(exc_type, exc_value, exc_tb)}
        '''
        print(strErrorMsg)

    # 關閉 sqlite
    conn.close()
    
# 下載
def download():
    # 寫入對話記錄
    conn = sqlite3.connect(f"{folderPath}/{sheetName}.db")
    
    # 將查詢出來的結果 (tuple)，變成 key-value 型式 (dict)
    conn.row_factory = sqlite3.Row
    
    # 建立 cursor 物件
    cursor = conn.cursor()

    # 執行 SQL 語法
    try:
        # 查詢特定資料
        sql_query = f'''
        SELECT sn, id, pdf_link
        FROM journals
        WHERE `is_downloaded` = 0
        '''
        
        # 更新資料的欄位(狀態)
        sql_update = f'''
        UPDATE `journals` 
        SET 
            `is_downloaded` = 1 ,
            `updated_at` = ?
        WHERE `id` = ?
        '''
            
        # 取得所有未下載的資料
        for myDict in cursor.execute(sql_query).fetchall():
            # 等待
            sleep(randint(1,2))

            # 下載 pdf
            cmd = ['curl', '-L', myDict["pdf_link"], '-o', f'{folderPath}/{myDict["sn"]}_{myDict["id"]}.pdf']
            result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            #output = result.stdout
            #pprint(output)
            print(f'{folderPath}/{myDict["sn"]}_{myDict["id"]}.pdf')

            # 將 is_downloaded 改成 1，代表已下載過
            cursor.execute(sql_update, (datetime.today().strftime("%Y-%m-%d %H-%M-%S"), myDict["id"],))
                
        conn.commit()
    except sqlite3.Error as err: 
        # 回滾
        conn.rollback()

        # SQLite3 例外處理
        exc_type, exc_value, exc_tb = sys.exc_info()
        strErrorMsg = f'''SQLite error: {' '.join(err.args)}\n\n
        SQLite traceback: {traceback.format_exception(exc_type, exc_value, exc_tb)}
        '''
        print(strErrorMsg)

    # 關閉 sqlite
    conn.close()

In [ ]:
# 初始化 Web Driver
init()

In [ ]:
# 走訪頁面
visit()

In [ ]:
# 剖析內容
parse()

In [ ]:
# 關閉瀏覽器
close()

In [ ]:
# 儲存成 json
saveJson()

In [ ]:
# 儲存 .db
saveDB()

In [ ]:
# 下載
download()